In [1]:
import sys
import tweepy
import sys
import tweepy
from nltk.corpus import stopwords
import re
from collections import Counter
import pandas as pd
import unidecode

consumer_key='YJDELxapXCz0dIbfJcJ27sfU5'
consumer_secret='q5ucGBB9xU9p5Xvorp9UwtvN3qaEhOlOFye3bL83YRRE9riTc0'
access_key='959019738855862273-PqPmgzE4fIrlLb9x49xSUAN5HoqjS15'
access_secret='jtYyvfdPiXGoeuJ5QnmnaAluw8bSitinnqUMI4Sl2cDWm'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


df = pd.read_csv('cities.csv', encoding='utf-8')

In [2]:
df.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Madrid,40.412752,-3.707721,Spain,ES,Madrid,primary,5567000.0,50437.0
1,Barcelona,41.398371,2.174100,Spain,ES,Catalonia,admin,4920000.0,1581595.0
2,Sevilla,37.382826,-5.973167,Spain,ES,Andalusia,admin,1212045.0,703021.0
3,Bilbao,43.260919,-2.938764,Spain,ES,Basque Country,minor,875552.0,353187.0
4,Valencia,39.466667,-0.366667,Spain,ES,Valencia,admin,808000.0,805304.0


In [3]:
Andalusia = ["Andalusia", "37.3399964,-4.5811614,250km"]
#Extremadura = 
Madrid = ["Madrid", "40.5248319,-3.7715628,60km"]
Catalonia = ["Catalonia", "41.8523094,1.5745043,150km"]
#CastillaLaMancha = 
#CastillaLeon = 
#Cantabria = 
Basque_Country = ["Basque Country", "42.9911816,-2.5543023,100km"]
#ComunidadValenciana = 
Canary_Islands = ["Canary Islands", "28.5306525,-15.7464439,400km"]
#Aragon = 
#LaRioja = 
#Navarra = 
#Asturias = 
#Murcia = 

CAS = [Andalusia, Madrid, Catalonia, Basque_Country, Canary_Islands]

In [4]:
#Rate limits are divided into 15 minute intervals. All endpoints require authentication, 
#so there is no concept of unauthenticated calls and rate limits. There are two initial buckets available 
#for GET requests: 15 calls every 15 minutes, and 180 calls every 15 minutes.

In [5]:
list(df[df["admin"] == "Canary Islands"]["city"])

['Santa Maria de Gracia',
 'Arrecife',
 'Las Palmas de Gran Canaria',
 'Santa Cruz de Tenerife']

In [6]:
CAS_ids = []
for CA in CAS:
    CA_cities = df[df["admin"] == CA[0]]["city"]
    tweets = tweepy.Cursor(api.search, geocode = CA[1]).items(300)  
    CA_ids = []
    for tweet in tweets:
        search_location = re.search('location\':(.+?),', str(tweet.user))
        if search_location:
            found_location = search_location.group(1)
            # Delete accent
            found_location = unidecode.unidecode(found_location)
            if any(word.lower() in found_location.lower() for word in CA_cities): 
                search_id = re.search('id\': (.+?),', str(tweet.user))
                if search_id:
                    found_id = search_id.group(1)
                    CA_ids.append(found_id)
    CAS_ids.append(CA_ids)


In [7]:
columns  =["Andalusia", "Madrid", "Catalonia", "Basque_Country", "Canary_Islands"]
df_ids = pd.DataFrame(data=list(zip(CAS_ids[0], CAS_ids[1], CAS_ids[2], CAS_ids[3], CAS_ids[4])),columns=columns)
print(df_ids)

              Andalusia               Madrid            Catalonia  \
0             392748212            225840482           3290824602   
1   1061369801434636288   922688039658426368  1034423944235954179   
2            2521085681           1013917093   748903179962687493   
3             599372052  1110843820579659777            333913500   
4             423706531   915803775507582976            139998032   
5   1086269060373929985            551119325            124932295   
6             367855881            609133159           1390080264   
7             633551251  1048917782253981696   975953621010481152   
8             875241254   907360620445618176   709843139314491392   
9            2237909977  1014255671192248320   986697120127348743   
10   706175784336171008            254179799             23606887   
11  1097866641566846976   722822285741019136            271090875   
12            324367893            262260222            254950846   
13            978808964           

In [8]:
df_ids.to_csv("CAS_ids2.csv")

In [ ]:
# Now do for each CA
tweet_users_text = []
for user_id in df_ids[CA[0]]:
    tweets = tweepy.Cursor(api.user_timeline, id=user_id).items(100)
    # Preprocess tweet
    tweet_text = [process_tweet(tweet.text) for tweet in tweets]
    tweet_users_text+=(tweet_text)

In [ ]:
def process_tweet(sentence):
    lower_case = sentence.lower()
    words = lower_case.split()
    words = [w for w in words if not w in stopwords.words("spanish")]
    words = [w for w in words if not "@" in w or if "rt" != w]     # Delete words that contain mentions or the RT word
    return sentence

In [ ]:
comunidades_tweets_clean = []
for sentence in comunidades_tweets[0]:
    lower_case = sentence.lower()
    words = lower_case.split()
    words = [w for w in words if not w in stopwords.words("spanish")]
    words = [w for w in words if not "@" in w or if "rt" != w]     # Delete words that contain mentions or the RT word
    comunidades_tweets_clean += words

In [ ]:
new_vals = Counter(comunidades_tweets_clean).most_common(10)

In [ ]:
new_vals